![logo](https://raw.githubusercontent.com/pdehidalgo/dbx-ucm-productivizacion/main/logo_ucm.jpg)

# Laboratorio: Evaluación de prompts con MLflow + Azure OpenAI + Telegram


**Objetivo**: Construir ChatBot con Telegram que permita describir imágenes técnicas recibidas mediante el cliente de Telegram.
Se deberán comparar múltiples prompts con MLflow, elegir el mejor (baśandose en un criterio a seleccionar razonadamente por el alumno) y luego usarlo para responder a imágenes enviadas desde un bot de Telegram.

## Requisitos de entrega:

- 1. Un script `main.py` funcional con el flujo completo
- 2. Una imagen/captura de pantalla de MLflow con los prompts registrados y [evaluados](https://mlflow.org/docs/latest/model-evaluation/)
- 3. Una imagen/captura de Telegram mostrando que se recibió una imagen y se respondió con la descripción del error

## Recursos disponibles

- Vídeo de la clase del Sábado 31 con la creación de un Servicio de Azure OpenAI para gpt-4o y Whispers. 
- Taller de mlflow para el despliegue en local en la documentación subida en Moodle. 
- Ejemplo de código funcional subido al repositorio del curso: ejemplo [end2end](https://github.com/pdehidalgo/dbx-ucm-productivizacion/blob/main/telegram_project/ejemplo_finalizado.py)

### Paso 1. Preparar entorno, MLflow y las variables y APIs necesarias: 

```python
import mlflow
import base64
from openai import AzureOpenAI
import os

# Configura tus variables de entorno (puedes usar dotenv si lo prefieres)
endpoint = os.getenv("OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")
deployment = "gpt-4o"
api_version = "2024-12-01-preview"

client = AzureOpenAI(
    api_key=api_key,
    api_version=api_version,
    azure_endpoint=endpoint,
)

```


### Paso 2. Definir prompts candidatos (al menos tres)

```python
prompts = [
    "Describe detalladamente lo que ves en la imagen, .....",
    "Eres un técnico especializado. ....",
    "Analiza visualmente la imagen y ..."
]

image_path = "sample_error.jpg"  # Usa una imagen simulada de error técnico (puede ser una captura de pantalla con mensaje de error)

```

### Paso 3. Evaluar cada prompt con gpt-4o y registrar en MLflow

```python
def evaluate_prompt(prompt, image_path):
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode()

    result = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    ### Incluye aquí la/s imágene/s
                ]
            }
        ]
    )
    return result.choices[0].message.content

mlflow.set_experiment("image-description-prompts")

with mlflow.start_run():
    for prompt in prompts:
        response = evaluate_prompt(prompt, image_path)
        
        # Incluye aquí tus métricas

        mlflow.log_param("prompt", prompt)
        mlflow.log_metric(...)

print("Evaluación de prompts completada. Verifica MLflow UI.")
```

**Tip**: el código anterior sirve de ejemplo, utiliza métricas de evaluación propias de mlflow.evaluate o crea tu propia métrica, piensa en las implicaciones de crear un dataset sintético y comparar la salida del sistema con tu descripción creada sobre el error de la imagen. 




### Paso 4. Obtener mejor prompt registrado

```python
def get_best_prompt():
    client = mlflow.tracking.MlflowClient()
    ## Crea tu criterio de ordenación
    runs = client.search_runs(experiment_ids=["0"], order_by=["metrics.clarity_score DESC"])
    return runs[0].data.params["prompt"]
```


### Paso 5. Usar mejor prompt para responder imágenes vía Telegram

Este código se incluirá en el archivo main.py junto con el bot. Usará el `get_best_prompt()` como prompt final.

Implementa un nuevo endpoint sobre el código de `ejemplo_finalizado.py` que permita interactuar con imágenes. No olvides restringir el uso...

```python
@bot.message_handler(content_types=["photo"])
def on_photo(message):
    user_id = message.chat.id
    file_info = bot.get_file(...Interactúa con la imagen...)
    file = requests.get(f"https://api.telegram.org/file/bot{os.getenv('TELEGRAM_BOT_TOKEN')}/{file_info.file_path}")

    
    # Llamada al asistente usando el mejor prompt y la imagen

    description = ...

    bot.send_message(user_id, f"Diagnóstico:\n{description}")

```



### Recuerda:
- Ejecuta `mlflow ui` en tu entorno para levantar la interfaz local
- Registra los resultados de cada prompt
- Usa `get_best_prompt()` para consultar el mejor prompt en tu bot Telegram

### Entrega esperada:
- main.py con flujo Telegram + OpenAI
- Imagen del dashboard MLflow mostrando parámetros y métricas
- Captura del bot Telegram recibiendo imagen y respondiendo diagnóstico
